In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from scipy import stats
from pprint import pprint
import sys
from colorama import Style, Fore
import warnings
import time 
import datetime as dt 


warnings.filterwarnings(action = 'ignore')

rc = {
    "axes.facecolor": "#F6F6F6",
    "figure.facecolor": "#F6F6F6",
    "axes.edgecolor": "#000000",
    "grid.color": "#EBEBE7",
    "font.family": "malgun gothic",
    "axes.labelcolor": "#000000",
    "xtick.color": "#000000",
    "ytick.color": "#000000",
    "grid.alpha": 0.4
}

sns.set(rc=rc)


red = Style.BRIGHT + Fore.RED
blu = Style.BRIGHT + Fore.BLUE
mgt = Style.BRIGHT + Fore.MAGENTA
gld = Style.BRIGHT + Fore.YELLOW
res = Style.RESET_ALL

In [31]:
data = pd.read_csv("C:/Users/whileduck/Desktop/Github/BigContest2023/data/2023빅콘테스트_어드밴스드리그_예술의전당.csv")

##### **콘서트홀이면서 경우로만 데이터 필터링 하기** 

In [32]:
cond = data['place'] == '콘서트홀'

data = data[cond]

In [33]:
data.head()

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,tran_date,tran_time,...,ticket_cancel,discount_type,performance_code,pre_open_date,open_date,genre,place,running_time,intermission,member_yn
0,50.0,F,골드,무료,그린,NaN,NaN,NaN,20220114,1512,...,2,일반,302,20220114.0,20220115.0,교향곡,콘서트홀,120,15,Y
4,50.0,M,무료,NaN,NaN,NaN,NaN,NaN,20220206,1615,...,0,일반,897,20220109.0,20220110.0,독주,콘서트홀,90,0,Y
5,30.0,F,블루,무료,NaN,NaN,NaN,NaN,20181124,1145,...,2,블루회원 할인20%,528,20181119.0,20181119.0,교향곡,콘서트홀,100,15,Y
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20191020,1459,...,0,초대권,44,NaN,20191023.0,교향곡,콘서트홀,100,15,N
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190613,954,...,0,초대권,827,20190602.0,20190602.0,교향곡,콘서트홀,120,20,N


# **빅 콘테스트 의문 사항**

### **1. price 가 0인 경우는 무엇일까 ? 초대권이 아님에도 불구하고 price 가 0인 경우도 있음**

In [51]:
df = data.copy()

price_cond = df['price'] == 0
price_0_df = df[price_cond]
total_discount_type = price_0_df['discount_type'].unique()
error_discount = ['기획사 할인50%', '기획사 판매20%','기획사 20%', '기획사할인40%','기획사 판매30%','연주자할인30%','기획사판매20%','기획사(특별할인)']
error_discount_cond = df['discount_type'].str.contains('|'.join(error_discount))

가격이0이면서_초대권이_아닌경우 = df[(price_cond) & (error_discount_cond)]

가격이0이면서_초대권이_아닌경우.to_csv('가격이0이면서_초대권이_아닌경우.csv',index = False)

In [52]:
length  = len(가격이0이면서_초대권이_아닌경우)

print('가격이 0이면서 초대권이 아닌 경우의 데이터 개수 : ',length)

가격이0이면서_초대권이_아닌경우.head()

가격이 0이면서 초대권이 아닌 경우의 데이터 개수 :  83


,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,tran_date,tran_time,...,ticket_cancel,discount_type,performance_code,pre_open_date,open_date,genre,place,running_time,intermission,member_yn
20705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20201025,1648,...,2,기획사 판매20%,1269,20200815.0,20200816.0,교향곡,콘서트홀,110,15,N
28797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190216,1738,...,0,기획사할인40%,2556,NaN,20190114.0,성악,콘서트홀,120,15,N
40211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20201025,1652,...,0,기획사 판매30%,1269,20200815.0,20200816.0,교향곡,콘서트홀,110,15,N
47478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20201025,1652,...,0,기획사 판매30%,1269,20200815.0,20200816.0,교향곡,콘서트홀,110,15,N
54771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190216,1738,...,0,기획사할인40%,2556,NaN,20190114.0,성악,콘서트홀,120,15,N


In [53]:
class DataExploratioin:
    '''
    데이터 탐색 시 사용 가능한 Class 

    기존 존재하는 프레임워크들을 이용하여 자주 이용하는 프레임워크들을 활용하여 나만의 분석 툴을 만들려고 함 

    데이터 요약, 결측값 처리 등의 내용이 담겨있는 class 
    '''

    def __init__(self, data):
        self.data = data

    def summarize(self):
        '''
        데이터를 초창기에 요약해주는 method
        '''

        cols = self.data.columns

        size = round(sys.getsizeof(self.data) / 1024 ** 2, 2)

        print(f'data size : {size}MB')

        self.result = pd.DataFrame()

        self.result['Dtype'] = self.data.dtypes.values
        self.result['Count'] = self.data.count().values
        self.result['Nunique'] = self.data.nunique().values
        self.result['Missing value'] = self.data.isna().sum().values
        self.result['Missing %'] = [str(round(
            missing / len(self.data), 2) * 100) + '%' for missing in self.result['Missing value']]
        self.result['Most Freq Value'] = self.data.mode().iloc[0].values

        freq_prop = []

        for i, col in enumerate(cols):

            raw_data = self.data.loc[~self.data[col].isna(), col]
            freq_value = self.result['Most Freq Value'].iloc[i]

            prop = np.mean(
                np.array(raw_data == freq_value)
            )

            prop_str = str(round(np.mean(prop) * 100, 1)) + '%'

            if prop_str == 'nan%':
                freq_prop.append(self.result['Missing %'].iloc[i])
            else:
                freq_prop.append(prop_str)

        self.result['Most Freq Value %'] = freq_prop

        self.result['Min'] = self.data.describe(include='all').T['min'].values
        self.result['Max'] = self.data.describe(include='all').T['max'].values
        self.result['Mean'] = self.data.describe(
            include='all').T['mean'].values
        self.result['Median'] = self.data.describe(
            include='all').T['50%'].values
        
        memory = (self.data.memory_usage(deep = True) // 1024 **2).values[1:] # index 의 usage 는 제외하고 보자 

        
        self.result['MB'] = [str(m) + ' mb' for m in memory]
        self.result = self.result.set_index(cols)

        self.result = self.result.fillna('-')

        display(self.result)
    
    
    def progress_bar(self,iterable, total_blocks = 10):
        
        total_items = len(iterable)
        block_size = total_items // total_blocks
        
        for i, item in enumerate(iterable, start=1):
            if i % block_size == 0 or i == total_items:
                progress = (i / total_items) * 100
                blocks = int(progress / (100 / total_blocks))
                empty_blocks = total_blocks - blocks
                progress_bar = '■' * blocks + '▢' * empty_blocks
                print(f"\rProgress: [{progress_bar}] {progress:.2f}%", end='', flush=True)
            yield item
            time.sleep(0.0000001)
    
    def reduce_size(self):
                
        original_size = round(sys.getsizeof(self.data) / 1024 ** 2,2)
        
        df = self.data.copy()
        
        for col in self.progress_bar(df.columns):
            
            dtp = df[col].dtype
            
            if dtp == 'object':
                df[col] = df[col].astype('category')
            else: # numeric type이면 
                
                if min(df[col]) >= 0 : # 부호가 없다면 unit 으로 변경해줘도 된다.
                    max_value = max(df[col])
                    
                    bits = [8,16,32,64]
                    
                    for bit in bits: # 최소한의 비트로 표현 될 수 있게 dtype 변경 
                        if max_value < 2 ** bit:
                            # 결측치가 있는 경우 astype 으로 변경하지 못하니 결측치를 채워준 후 변경하고 다시 결측치를 채우자 
                            df[col] = df[col].fillna(2 ** bit - 1)
                            df[col] = df[col].astype(f'uint{bit}')
                            df[col] = df[col].replace(2 ** bit - 1, np.NaN)
                            break
                        
                else: # 부호가 있다면 int type 으로 바꿔주자 
                    
                    max_value = max(abs(min(df[col])), max(df[col]))
                    
                    bits = [8,16,32,64]
                    
                    for bit in bits:
                        if max_value < 2 ** bit:
                            df[col] = df[col].fillna(2 ** bit - 1)
                            df[col] = df[col].astype(f'int{bit}')
                            df[col] = df[col].replace(2 ** bit - 1, np.NaN)
                            break
                        
        print('\n')
                        
        after_size = round(sys.getsizeof(df) / 1024 ** 2,2)
        
        # 바꾼 후 결과 보여주기 
        after = DataExploratioin(df)
        after.summarize()
        
        print(f'\n {original_size}MB -> {after_size}MB')
            
        return df
                        
            
                

In [54]:
ep = DataExploratioin(data)

In [55]:
ep.summarize()

data size : 886.02MB


,Dtype,Count,Nunique,Missing value,Missing %,Most Freq Value,Most Freq Value %,Min,Max,Mean,Median,MB
age,float64,484517,8,612451,56.00000000000001%,50.0,23.8%,10.0,80.0,43.018718,40.0,8 mb
gender,object,484867,2,612101,56.00000000000001%,F,65.1%,-,-,-,-,45 mb
membership_type_1,object,484867,3,612101,56.00000000000001%,무료,47.5%,-,-,-,-,58 mb
membership_type_2,object,389737,5,707231,64.0%,무료,49.7%,-,-,-,-,53 mb
membership_type_3,object,140468,4,956500,87.0%,그린,43.6%,-,-,-,-,40 mb
membership_type_4,object,25160,3,1071808,98.0%,그린,77.7%,-,-,-,-,34 mb
membership_type_5,object,1110,1,1095858,100.0%,그린,100.0%,-,-,-,-,33 mb
membership_type_6,float64,0,0,1096968,100.0%,-,100.0%,-,-,-,-,8 mb
tran_date,int64,1096968,1687,0,0.0%,20221112,0.7%,20181101.0,20230703.0,20210083.786885,20211107.0,8 mb
tran_time,int64,1096968,1440,0,0.0%,1401,0.7%,0.0,2359.0,1384.013409,1412.0,8 mb



# **💡insight** 

데이터 사이즈 : 841.91 MB


0. 전체 데이터 데이터 중 빈좌석은 56% 에 육박합니다. 코로나로 인한 대면 거리두기 등에 따라 상이하겠지만 age 와 gender 의 경우가 missing value 인 경우가 빈좌석입니다.

1. 연령대는 bining 된 형태로 존재합니다 10대~80대 까지 있습니다. 현재 테이블에서 50대 연령이 56% 가량 있습니다.

2. 성별은 여성의 비율이 65%, 남성의 비율이 35%입니다

3. 각 멤버십 타입 별 멤버십 가지수의 최대치는 최대 5개인 것으로 보입니다. 멤버십 타입 5, 6 까지 가지고 결제한 고객은 많이 있지 않습니다.

4. 유니크한 좌석의 개수는 2509개입니다.

5. 가격들의 유니크한 개수는 206 개입니다. 가격이 0인 경우가 전체 데이터에서 52.8%나. 됩니다. 좌석이 판매되지 않았거나, 초대권 혹은 무료 입장권등으로 보입니다.

6. 예매 취소 비율은 26.6% 입니다. (2 는 예매 취소하지 않음, 0은 예매 취소하지 않음)

7. 할인을 받은 경우 초대권이 35.4%로 가장 많았습니다. 

8. 공연 장르의 유니크한 개수는 13개입니다. 그 중 교향곡이 48.7% 로 가장 빈도가 높았습니다. 

9. 공연 장르가 결측치인 경우가 있습니다. 공연 장르가 결측치인 경우 정확한 데이터가 아니라고 판단하여 드롭시키겠습니다.


In [56]:
# 장르가 결측치인 경우는 drop 

df = df[df['genre'].notna()]

In [59]:
ep = DataExploratioin(df)

### **데이터 용량 줄이기**

In [60]:
df = ep.reduce_size()

Progress: [■■■■■■■■■■] 100.00%

data size : 72.04MB


,Dtype,Count,Nunique,Missing value,Missing %,Most Freq Value,Most Freq Value %,Min,Max,Mean,Median,MB
age,float64,473767,8,598128,56.00000000000001%,50.0,23.8%,10.0,80.0,42.914027,40.0,8 mb
gender,category,474088,2,597807,56.00000000000001%,F,64.8%,-,-,-,-,1 mb
membership_type_1,category,474088,3,597807,56.00000000000001%,무료,47.5%,-,-,-,-,1 mb
membership_type_2,category,381697,5,690198,64.0%,무료,49.7%,-,-,-,-,1 mb
membership_type_3,category,138007,4,933888,87.0%,그린,43.8%,-,-,-,-,1 mb
membership_type_4,category,24802,3,1047093,98.0%,그린,77.9%,-,-,-,-,1 mb
membership_type_5,category,1104,1,1070791,100.0%,그린,100.0%,-,-,-,-,1 mb
membership_type_6,float64,0,0,1071895,100.0%,-,100.0%,-,-,-,-,8 mb
tran_date,uint32,1071895,1687,0,0.0%,20230506,0.7%,20181101.0,20230703.0,20209811.927451,20211029.0,4 mb
tran_time,uint16,1071895,1440,0,0.0%,1401,0.6%,0.0,2359.0,1384.900139,1413.0,2 mb



 867.26MB -> 72.04MB


# **💡insight** 

데이터 타입을 변경하는 것만으로도 용량을 90% 가량 줄일 수 있었습니다.

데이터 타입을 변경하여도 데이터의 통계량에는 변화가 없는 모습을 볼 수 있습니다.

데이터의 메모리 사용량을 감소시켜 이후 과정에서 연산속도의 향상을 기대 할 수 있을 것 같습니다.

### **시계열 데이터 처리**

| 컬럼ID | 컬럼명 | 비고 |
|-------|-------|-------|
| tran_date   |예매 거래 일자    | YYYYMMDD   |
| tran_time   |예매 거래 시간  |HHMM   |
| play_date   |공연 시작 날짜  |YYYYMMDD   |
| play_st_time   |공연 시작 시간  |HHMM   |
| pre_open_date    |선예매시작일  |YYYYMMDD   |
| open_date    |예매시작일  |YYYYMMDD   |


시계열 데이터를 데이터 타임 형태로 변경해주겠습니다.

pandas 의 pd.to_datetime 을 이용하면 쉽게 변경 할 수 있습니다.

In [61]:
def change_datetiime(df, cols):
    
    format_type = {
        'tran_date' : 'YYYYMMDD',
        'tran_time' : 'HHMM',
        'play_date' : 'YYYYMMDD',
        'play_st_time' : 'HHMM',
        'pre_open_date' : 'YYYYMMDD',
        'open_date' : 'YYYYMMDD'
    }
    
    data = df.copy()
    
    for col in cols:
        
        if format_type[col] == 'YYYYMMDD':
            format = '%Y%m%d'
            
            data[col] = pd.to_datetime(data[col], format = format)
            
        if format_type[col] == 'HHMM':
            format = '%H%M'

            fill_col = data[col].apply(lambda x: str(x).zfill(4)) # 문자열 형태로 변경 후 4글자인 HHMM 형태로 맞춰줍니다.
                                                            # 현재 데이터에서는 오전 9시 30분인 경우 930 으로 써있습니다.
            data[col] = pd.to_datetime(fill_col, format = format).dt.time
    return data

In [62]:
time_cols = ['tran_date' , 'tran_time','play_date', 'play_st_time','pre_open_date','open_date']

df = change_datetiime(df,  cols = time_cols )

display(df[time_cols])

,tran_date,tran_time,play_date,play_st_time,pre_open_date,open_date
0,2022-01-14,15:12:00,2022-02-04,20:00:00,2022-01-14,2022-01-15
4,2022-02-06,16:15:00,2022-03-02,19:30:00,2022-01-09,2022-01-10
5,2018-11-24,11:45:00,2019-03-23,20:00:00,2018-11-19,2018-11-19
6,2019-10-20,14:59:00,2019-11-09,17:00:00,NaT,2019-10-23
7,2019-06-13,09:54:00,2019-07-23,20:00:00,2019-06-02,2019-06-02
...,...,...,...,...,...,...
1920853,2021-06-18,15:09:00,2021-07-04,15:00:00,2021-06-18,2021-06-20
1920855,2023-05-22,17:29:00,2023-06-13,17:00:00,2023-05-05,2023-05-07
1920856,2020-10-09,16:52:00,2020-10-20,19:30:00,NaT,2020-09-20
1920858,2020-07-26,16:55:00,2020-08-18,19:30:00,2020-07-19,2020-07-20


In [63]:
# 시계열 데이터  전처리

df['공연연도'] = df['play_date'].dt.year
df['공연월'] = df['play_date'].dt.month
df['공연일'] = df['play_date'].dt.date
df['공연연월'] = df['공연연도'].astype(str) + '-' + df['공연월'].astype(str)
df['공연연월'] = pd.to_datetime(df['공연연월'])
df['전체공연시간'] = pd.to_datetime(df['공연일'].astype(str) + ' ' + df['play_st_time'].astype(str))
df['전체거래시간'] = pd.to_datetime(df['tran_date'].astype(str) + ' ' + df['tran_time'].astype(str))

# **좌석 전처리**

In [64]:
df['층'] = df['seat'].map(lambda x: x.split()[0])
df['석'] = df['seat'].map(lambda x: x.split()[1])
df['세부좌석'] = df['seat'].map(lambda x: x.split()[2])

In [65]:
# 전처리 내용 확인 
arr = ['층','석','세부좌석']

for a in arr:
    
    uniq = sorted(df[a].unique())
    
    print(f'{a}의 유니크한 값들')
    
    if len(uniq) <= 5:
        print(uniq)
    else:
        for i in range(0,len(uniq),10):
            print(uniq[i:i+10])

층의 유니크한 값들
['1층', '2층', '3층', '합창석']
석의 유니크한 값들
['A블록10열', 'A블록11열', 'A블록12열', 'A블록13열', 'A블록15열', 'A블록16열', 'A블록17열', 'A블록18열', 'A블록19열', 'A블록1열']
['A블록20열', 'A블록21열', 'A블록22열', 'A블록2열', 'A블록3열', 'A블록4열', 'A블록5열', 'A블록6열', 'A블록7열', 'A블록8열']
['A블록9열', 'BOX1', 'BOX10', 'BOX11', 'BOX12', 'BOX2', 'BOX3', 'BOX4', 'BOX5', 'BOX6']
['BOX7', 'BOX8', 'BOX9', 'B블록10열', 'B블록11열', 'B블록12열', 'B블록13열', 'B블록14열', 'B블록15열', 'B블록16열']
['B블록17열', 'B블록18열', 'B블록19열', 'B블록1열', 'B블록20열', 'B블록21열', 'B블록22열', 'B블록2열', 'B블록3열', 'B블록4열']
['B블록5열', 'B블록6열', 'B블록7열', 'B블록8열', 'B블록9열', 'C블록10열', 'C블록11열', 'C블록12열', 'C블록13열', 'C블록14열']
['C블록15열', 'C블록16열', 'C블록17열', 'C블록18열', 'C블록19열', 'C블록1열', 'C블록20열', 'C블록21열', 'C블록22열', 'C블록2열']
['C블록3열', 'C블록4열', 'C블록5열', 'C블록6열', 'C블록7열', 'C블록8열', 'C블록9열', 'D블록10열', 'D블록11열', 'D블록12열']
['D블록13열', 'D블록14열', 'D블록15열', 'D블록16열', 'D블록17열', 'D블록18열', 'D블록19열', 'D블록1열', 'D블록20열', 'D블록21열']
['D블록22열', 'D블록2열', 'D블록3열', 'D블록4열', 'D블록5열', 'D블록6열', 'D블록7열', 'D블록8열', 'D블록9열', 'E블록10열']

층, 좌석, 세부좌석에 대한 유니크한 값이 잘 처리 된 것을 보입니다.

In [66]:
print(df.shape)
df.head()

(1071895, 33)


,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,tran_date,tran_time,...,member_yn,공연연도,공연월,공연일,공연연월,전체공연시간,전체거래시간,층,석,세부좌석
0,50.0,F,골드,무료,그린,NaN,NaN,NaN,2022-01-14,15:12:00,...,Y,2022,2,2022-02-04,2022-02-01,2022-02-04 20:00:00,2022-01-14 15:12:00,3층,BOX9,10
4,50.0,M,무료,NaN,NaN,NaN,NaN,NaN,2022-02-06,16:15:00,...,Y,2022,3,2022-03-02,2022-03-01,2022-03-02 19:30:00,2022-02-06 16:15:00,1층,B블록12열,7
5,30.0,F,블루,무료,NaN,NaN,NaN,NaN,2018-11-24,11:45:00,...,Y,2019,3,2019-03-23,2019-03-01,2019-03-23 20:00:00,2018-11-24 11:45:00,1층,A블록2열,1
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-10-20,14:59:00,...,N,2019,11,2019-11-09,2019-11-01,2019-11-09 17:00:00,2019-10-20 14:59:00,3층,E블록4열,8
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-06-13,09:54:00,...,N,2019,7,2019-07-23,2019-07-01,2019-07-23 20:00:00,2019-06-13 09:54:00,2층,D블록8열,4


In [67]:
df.to_csv('dataframe_reduced_size.csv',index = False)